In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import matplotlib
from matplotlib import pyplot as plt
# import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [78]:
dataFull = pd.read_csv("../Instagram_Like_Predictor/InstagramDatasetFinalVersion.csv",index_col = 0)
dataFull.head()
dataText = dataFull.loc[:,['hashtags','mentions','location','edge_media_to_caption']]
dataLabels = dataFull.loc[:,['mean_likes']]

dataText['mentions'].replace('[]','',inplace=True)
dataText['hashtags'].replace('[]', '', inplace=True)
#dataText['location'] = dataText.fillna({'location':''})

dataText.fillna({'location':''},inplace=True)
dataText.fillna({'edge_media_to_caption':''},inplace = True)
#replaced nontext with symbols, need to clear it out.

dataText.head()
dataText.to_csv('removeNull.csv', index=None, header= True)
dataText.info()



<class 'pandas.core.frame.DataFrame'>

📧takashi.yasui1980@gmail.com
Data columns (total 4 columns):
hashtags                 20668 non-null object
mentions                 20668 non-null object
location                 20668 non-null object
edge_media_to_caption    20668 non-null object
dtypes: object(4)
memory usage: 807.3+ KB


In [74]:
# bag of words, tfidf, autoencoder, 
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)
dataText['edge_media_to_caption'] = data['edge_media_to_caption'].apply(remove_punctuation)


AttributeError: 'float' object has no attribute 'translate'